In [305]:
import numpy as np

def fast_marching_2D(W, x0):
    n = W.shape[0]
    neigh = np.array([[1,0],[-1,0],[0,1],[0,-1]])
    boundary = lambda x: x*np.logical_and(x<=n,x>0) + (2-x) * (x<=0) + (2*n-x)*(x>n)
    ind2sub1 = lambda k: [(k-1)%n +1, (k- (k-1)%n -1)/n +1]
    sub2ind1 = lambda u: (u[1]-1)*n + u[0]
    
    Neigh = lambda k,i: sub2ind1(boundary(ind2sub1(k)+neigh[i,:]))

    I = np.atleast_1d(sub2ind1(x0))

    D = np.zeros((n,n)) + np.inf
    D[np.unravel_index(I, D.shape, 'F')] = 0

    S = np.zeros((n,n))
    S[np.unravel_index(I, S.shape, 'F')] =1

    while np.any(I):
        if isinstance(D[np.unravel_index(I, D.shape, 'F')],float):
            [tmp,j]=[D[np.unravel_index(I, D.shape, 'F')],0]
            j = 0
            i = I
            I = np.array([])
        
        else:
            [tmp,j] = [np.sort(D[np.unravel_index(I, D.shape, 'F')]),np.argsort(D[np.unravel_index(I, D.shape, 'F')])]

            j = j[0]
            i = I[j]
            np.delete(I,j)
        S[np.unravel_index(i, S.shape, 'F')] = -1

        J = np.array([Neigh(i,0),Neigh(i,1),Neigh(i,2), Neigh(i,3)],dtype=np.int16)

        #J[S[np.unravel_index(J, S.shape, 'F')]==-1] = []
        np.delete(J,S[np.unravel_index(J, S.shape, 'F')]==-1)
        
        J1 = J[S[np.unravel_index(J, S.shape, 'F')]==0]
        I = J1
        S[np.unravel_index(J1, S.shape, 'F')]=1
        
        for j in J:
            dx = np.min(D[np.unravel_index([int(Neigh(j,0)),int(Neigh(j,1))], D.shape, 'F')])
            dy = np.min(D[np.unravel_index([int(Neigh(j,2)),int(Neigh(j,3))], D.shape, 'F')])
            Delta = 2*W[np.unravel_index(j,W.shape,'F')] - (dx-dy)**2
            if Delta >=0:
                D[np.unravel_index(j, D.shape, 'F')] = (dx+dy+np.sqrt(Delta))/2
            else:
                D[np.unravel_index(j,D.shape,'F')] = min(dx+W[np.unravel_index(j,W.shape,'F')], dy+W[np.unravel_index(j,W.shape,'F')])
                     
    S = np.zeros((n,n))

    return D, S

In [312]:
from imageio import imread
img = imread('vessels.png')
D,S = fast_marching_2D(img, [142,226])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]]


ValueError: index -10043 is out of bounds for array with size 60516

In [309]:
import matplotlib.pyplot as plt
print(D)

[[inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]]


In [122]:
a = np.zeros((3,3))
inds = [1,2,3,4]

a[np.unravel_index(inds, a.shape, 'F')] = 1
#   array([1, 4, 7, 2, 5])

In [123]:
a

array([[0., 1., 0.],
       [1., 1., 0.],
       [1., 0., 0.]])

In [115]:
a.flatten()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [311]:
np.any(a)

True